In [11]:
from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import load_account, get_token_authentication, get_basic_authentication
from quantuminspire.api import QuantumInspireAPI
# from quantuminspire.projectq.backend_qx import QIBackend

import numpy as np
import pandas as pd

In [12]:
print('Enter mail address')
email = input()

print('Enter password')
password = getpass()

server_url = r'https://api.quantum-inspire.com'
authentication = BasicAuthentication(email, password)
qi = QuantumInspireAPI(server_url, authentication)

Enter mail address
francesco.zatelli@gmail.com
Enter password
········


In [13]:
def sphere_meshing(n_θ, n_ϕ):
    '''
    n_θ: partition number for polar angle
    n_ϕ: partition number for azimuthal angle
    '''
    coords=[]
    set_θ=np.linspace(0,np.pi,n_θ)
    set_ϕ=np.linspace(0,2*np.pi,n_ϕ)
    for θ in set_θ:
        if(θ==0 or θ==2*np.pi):
            coords.append((θ,0))
        else:
            for ϕ in set_ϕ:
                coords.append((θ,ϕ))
    return coords

In [20]:
import plotly.express as px

#https://stackoverflow.com/questions/9600801/evenly-distributing-n-points-on-a-sphere

num_pts = 1000
indices = np.arange(0, num_pts, dtype=float) + 0.5

phi = np.arccos(1 - 2*indices/num_pts)
theta = (np.pi * (1 + 5**0.5) * indices)%(2*np.pi)
coords=[]

for theta_elem,phi_elem in zip(theta,phi):
    coords.append((theta_elem,phi_elem))

x, y, z = np.cos(theta) * np.sin(phi), np.sin(theta) * np.sin(phi), np.cos(phi);

# Plot the surface.
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, template="plotly_white")

fig.show()

In [21]:
def parameterized_QACM(loc):
    θ,ϕ=loc
    qasm = '''
    version 1.0

    qubits 5

    # initialize the state
    prep_z q[2]
    Ry q[2], {0}
    Rz q[2], {1}
    

    #preparation
    Ry q[0], 1.107149
    #rewrite CNOT q[0],q[4] and CNOT q[4],q[0] usign nearest neighbors
    SWAP q[0],q[2] 
    CNOT q[2],q[4]

    Ry q[4], 0.729728
    CNOT q[4],q[2]
    SWAP q[0],q[2]

    Ry q[0], 0.463648

    #copying
    CNOT q[2], q[0]
    CNOT q[2], q[4]
    CNOT q[0], q[2]
    CNOT q[4], q[2]

    #Rotate back and measure
    Rz q[0], {2}
    Ry q[0], {3}
    Measure_z q[0]
    '''
    return qasm.format(θ,ϕ,-ϕ,-θ)

In [23]:
npoints=1000
N_shots=4096
target_points=np.array(coords)
# save the data, optional
np.savetxt("target_points_starmon.csv",target_points)

In [24]:
# iterate over the sphere, run task
backend_type = qi.get_backend_type_by_name('Starmon-5')

copied_data=[]
index=0
for points in target_points:
    qasm=parameterized_QACM(points)
    result = qi.execute_qasm(qasm, backend_type=backend_type, number_of_shots=N_shots)
    hist=result.get('histogram', {})
    copied_data.append([hist["0"],hist["1"]])
    index=index+1
    print(str(index)+"/"+str(npoints))
    print(points)
    print([hist["0"],hist["1"]])
copied_data=np.array(copied_data)


1/1000
[5.08320369 0.04472509]
[0.570068359375, 0.429931640625]
2/1000
[2.68324046 0.07747904]
[0.594970703125, 0.405029296875]
3/1000
[0.28327723 0.10004171]
[0.698486328125, 0.301513671875]
4/1000
[4.16649931 0.11839073]
[0.511474609375, 0.488525390625]
5/1000
[1.76653608 0.13426491]
[0.60986328125, 0.39013671875]
6/1000
[5.64975816 0.14846027]
[0.654052734375, 0.345947265625]
7/1000
[3.24979493 0.16142035]
[0.59423828125, 0.40576171875]
8/1000
[0.8498317  0.17342232]
[0.66943359375, 0.33056640625]
9/1000
[4.73305378 0.18465311]
[0.564697265625, 0.435302734375]
10/1000
[2.33309055 0.19524586]
[0.61865234375, 0.38134765625]
11/1000
[6.21631262 0.20529936]
[0.712646484375, 0.287353515625]
12/1000
[3.81634939 0.21488933]
[0.55859375, 0.44140625]
13/1000
[1.41638616 0.22407529]
[0.62841796875, 0.37158203125]
14/1000
[5.29960824 0.23290506]
[0.62548828125, 0.37451171875]
15/1000
[2.89964501 0.24141773]
[0.6005859375, 0.3994140625]
16/1000
[0.49968178 0.24964577]
[0.704345703125, 0.2956542

131/1000
[0.96406388 0.73921151]
[0.716064453125, 0.283935546875]
132/1000
[4.84728596 0.74217535]
[0.633544921875, 0.366455078125]
133/1000
[2.44732273 0.74512965]
[0.66015625, 0.33984375]
134/1000
[0.0473595  0.74807451]
[0.75244140625, 0.24755859375]
135/1000
[3.93058157 0.75101006]
[0.5732421875, 0.4267578125]
136/1000
[1.53061835 0.75393641]
[0.69873046875, 0.30126953125]
137/1000
[5.41384042 0.75685366]
[0.695556640625, 0.304443359375]
138/1000
[3.01387719 0.75976193]
[0.62548828125, 0.37451171875]
139/1000
[0.61391396 0.76266133]
[0.72802734375, 0.27197265625]
140/1000
[4.49713604 0.76555195]
[0.614013671875, 0.385986328125]
141/1000
[2.09717281 0.76843391]
[0.67626953125, 0.32373046875]
142/1000
[5.98039489 0.7713073 ]
[0.727783203125, 0.272216796875]
143/1000
[3.58043166 0.77417222]
[0.57080078125, 0.42919921875]
144/1000
[1.18046843 0.77702877]
[0.7158203125, 0.2841796875]
145/1000
[5.06369051 0.77987705]
[0.678466796875, 0.321533203125]
146/1000
[2.66372728 0.78271715]
[0.64

259/1000
[1.64485053 1.0667187 ]
[0.68603515625, 0.31396484375]
260/1000
[5.5280726  1.06900136]
[0.724365234375, 0.275634765625]
261/1000
[3.12810937 1.07128116]
[0.607666015625, 0.392333984375]
262/1000
[0.72814614 1.07355813]
[0.718017578125, 0.281982421875]
263/1000
[4.61136822 1.0758323 ]
[0.625244140625, 0.374755859375]
264/1000
[2.21140499 1.07810367]
[0.6826171875, 0.3173828125]
265/1000
[6.09462707 1.08037228]
[0.749267578125, 0.250732421875]
266/1000
[3.69466384 1.08263814]
[0.583740234375, 0.416259765625]
267/1000
[1.29470061 1.08490128]
[0.71484375, 0.28515625]
268/1000
[5.17792269 1.08716172]
[0.6904296875, 0.3095703125]
269/1000
[2.77795946 1.08941947]
[0.627685546875, 0.372314453125]
270/1000
[0.37799623 1.09167457]
[0.72802734375, 0.27197265625]
271/1000
[4.26121831 1.09392703]
[0.600341796875, 0.399658203125]
272/1000
[1.86125508 1.09617687]
[0.6953125, 0.3046875]
273/1000
[5.74447715 1.09842411]
[0.748291015625, 0.251708984375]
274/1000
[3.34451392 1.10066878]
[0.6101

388/1000
[6.20885925 1.34385329]
[0.725830078125, 0.274169921875]
389/1000
[3.80889602 1.34590544]
[0.58837890625, 0.41162109375]
390/1000
[1.40893279 1.34795662]
[0.68212890625, 0.31787109375]
391/1000
[5.29215487 1.35000685]
[0.717529296875, 0.282470703125]
392/1000
[2.89219164 1.35205614]
[0.616943359375, 0.383056640625]
393/1000
[0.49222841 1.3541045 ]
[0.69873046875, 0.30126953125]
394/1000
[4.37545049 1.35615193]
[0.61865234375, 0.38134765625]
395/1000
[1.97548726 1.35819845]
[0.65771484375, 0.34228515625]
396/1000
[5.85870933 1.36024406]
[0.73681640625, 0.26318359375]
397/1000
[3.4587461  1.36228878]
[0.590087890625, 0.409912109375]
398/1000
[1.05878287 1.36433262]
[0.664306640625, 0.335693359375]
399/1000
[4.94200495 1.36637558]
[0.667236328125, 0.332763671875]
400/1000
[2.54204172 1.36841768]
[0.650390625, 0.349609375]
401/1000
[0.14207849 1.37045892]
[0.71630859375, 0.28369140625]
402/1000
[4.02530057 1.37249932]
[0.596923828125, 0.403076171875]
403/1000
[1.62533734 1.3745388

516/1000
[0.60646059 1.60180129]
[0.692138671875, 0.307861328125]
517/1000
[4.48968267 1.60380232]
[0.635986328125, 0.364013671875]
518/1000
[2.08971944 1.60580348]
[0.665283203125, 0.334716796875]
519/1000
[5.97294151 1.60780477]
[0.7431640625, 0.2568359375]
520/1000
[3.57297828 1.60980622]
[0.59521484375, 0.40478515625]
521/1000
[1.17301505 1.61180782]
[0.6875, 0.3125]
522/1000
[5.05623713 1.61380959]
[0.6923828125, 0.3076171875]
523/1000
[2.6562739  1.61581153]
[0.635498046875, 0.364501953125]
524/1000
[0.25631067 1.61781365]
[0.73779296875, 0.26220703125]
525/1000
[4.13953275 1.61981596]
[0.59912109375, 0.40087890625]
526/1000
[1.73956952 1.62181846]
[0.673583984375, 0.326416015625]
527/1000
[5.6227916  1.62382117]
[0.741943359375, 0.258056640625]
528/1000
[3.22282837 1.62582409]
[0.6025390625, 0.3974609375]
529/1000
[0.82286514 1.62782724]
[0.686279296875, 0.313720703125]
530/1000
[4.70608722 1.62983061]
[0.66650390625, 0.33349609375]
531/1000
[2.30612399 1.63183422]
[0.6452636718

645/1000
[5.17046931 1.86397843]
[0.71875, 0.28125]
646/1000
[2.77050608 1.86606823]
[0.641357421875, 0.358642578125]
647/1000
[0.37054285 1.86815937]
[0.71484375, 0.28515625]
648/1000
[4.25376493 1.87025185]
[0.59423828125, 0.40576171875]
649/1000
[1.8538017  1.87234567]
[0.63818359375, 0.36181640625]
650/1000
[5.73702378 1.87444087]
[0.73681640625, 0.26318359375]
651/1000
[3.33706055 1.87653744]
[0.610107421875, 0.389892578125]
652/1000
[0.93709732 1.8786354 ]
[0.667236328125, 0.332763671875]
653/1000
[4.8203194  1.88073476]
[0.65380859375, 0.34619140625]
654/1000
[2.42035617 1.88283553]
[0.634765625, 0.365234375]
655/1000
[0.02039294 1.88493772]
[0.730712890625, 0.269287109375]
656/1000
[3.90361501 1.88704136]
[0.570068359375, 0.429931640625]
657/1000
[1.50365179 1.88914644]
[0.635498046875, 0.364501953125]
658/1000
[5.38687386 1.89125298]
[0.719970703125, 0.280029296875]
659/1000
[2.98691063 1.893361  ]
[0.63916015625, 0.36083984375]
660/1000
[0.5869474 1.8954705]
[0.691650390625, 

774/1000
[3.45129273 2.14957269]
[0.606201171875, 0.393798828125]
775/1000
[1.0513295  2.15196367]
[0.637451171875, 0.362548828125]
776/1000
[4.93455158 2.15435841]
[0.678955078125, 0.321044921875]
777/1000
[2.53458835 2.15675694]
[0.6259765625, 0.3740234375]
778/1000
[0.13462512 2.15915929]
[0.74267578125, 0.25732421875]
779/1000
[4.0178472  2.16156551]
[0.585205078125, 0.414794921875]
780/1000
[1.61788397 2.16397561]
[0.587646484375, 0.412353515625]
781/1000
[5.50110604 2.16638963]
[0.727294921875, 0.272705078125]
782/1000
[3.10114281 2.16880761]
[0.62646484375, 0.37353515625]
783/1000
[0.70117958 2.17122958]
[0.661865234375, 0.338134765625]
784/1000
[4.58440166 2.17365557]
[0.6279296875, 0.3720703125]
785/1000
[2.18443843 2.17608562]
[0.62353515625, 0.37646484375]
786/1000
[6.06766051 2.17851977]
[0.729736328125, 0.270263671875]
787/1000
[3.66769728 2.18095804]
[0.603515625, 0.396484375]
788/1000
[1.26773405 2.18340047]
[0.60107421875, 0.39892578125]
789/1000
[5.15095613 2.18584711]

902/1000
[4.13207938 2.50310834]
[0.577880859375, 0.422119140625]
903/1000
[1.73211615 2.5064718 ]
[0.515380859375, 0.484619140625]
904/1000
[5.61533822 2.50985067]
[0.704833984375, 0.295166015625]
905/1000
[3.21537499 2.51324521]
[0.59521484375, 0.40478515625]
906/1000
[0.81541176 2.51665569]
[0.5966796875, 0.4033203125]
907/1000
[4.69863384 2.52008237]
[0.6025390625, 0.3974609375]
908/1000
[2.29867061 2.52352552]
[0.57275390625, 0.42724609375]
909/1000
[6.18189269 2.52698543]
[0.713623046875, 0.286376953125]
910/1000
[3.78192946 2.53046238]
[0.56396484375, 0.43603515625]
911/1000
[1.38196623 2.53395668]
[0.547607421875, 0.452392578125]
912/1000
[5.26518831 2.53746862]
[0.67529296875, 0.32470703125]
913/1000
[2.86522508 2.54099853]
[0.599609375, 0.400390625]
914/1000
[0.46526185 2.54454672]
[0.65283203125, 0.34716796875]
915/1000
[4.34848393 2.54811352]
[0.56689453125, 0.43310546875]
916/1000
[1.9485207  2.55169929]
[0.53515625, 0.46484375]
917/1000
[5.83174277 2.55530436]
[0.71215820

In [25]:
data_sheet=pd.DataFrame(data=np.hstack((target_points,copied_data)),columns=["θ","ϕ","prob_0","prob_1"])

In [26]:
data_sheet.head()

,θ,ϕ,prob_0,prob_1
0,5.083204,0.044725,0.570068,0.429932
1,2.683240,0.077479,0.594971,0.405029
2,0.283277,0.100042,0.698486,0.301514
3,4.166499,0.118391,0.511475,0.488525
4,1.766536,0.134265,0.609863,0.390137


In [27]:
# save to excel, optional
data_sheet.to_excel("data_sheet_starmon.xlsx")

In [28]:
data_sheet=pd.read_excel("data_sheet_starmon.xlsx")

In [29]:
def spherical2cartesian (θ,ϕ):
    x=np.sin(θ)*np.cos(ϕ)
    y=np.sin(θ)*np.sin(ϕ)
    z=np.cos(θ)
    return x,y,z

In [32]:
# Make data.
thetas=data_sheet["θ"]
phis=data_sheet["ϕ"]
x, y, z = np.cos(thetas) * np.sin(phis), np.sin(thetas) * np.sin(phis), np.cos(phis);

# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0/(5/6), template="plotly_white")

fig.show()

